# Runtime Test

Before runing this test, please download models from [https://nomeroff.net.ua/models/](https://nomeroff.net.ua/models/) to **./models/** directory. 

In [1]:
# Specify device
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "" 

In [3]:
import os
import sys
import json
import numpy as np
import cv2
from matplotlib import pyplot as plt
from termcolor import colored
import warnings
import time
warnings.filterwarnings('ignore')

# change this property
NOMEROFF_NET_DIR = os.path.abspath('../')
sys.path.append(NOMEROFF_NET_DIR)

In [5]:
# Import license plate recognition tools.
from NomeroffNet import  filters
from NomeroffNet import  RectDetector
from NomeroffNet import  TextDetector
from NomeroffNet import  OptionsDetector
from NomeroffNet.DetectronDetector import  Detector
from NomeroffNet import  textPostprocessing
from NomeroffNet import  textPostprocessingAsync

rectDetector = RectDetector()

optionsDetector = OptionsDetector()
optionsDetector.load("latest")

# Initialize text detector.
textDetector = TextDetector({
    "eu_ua_2004_2015": {
        "for_regions": ["eu_ua_2015", "eu_ua_2004"],
        "model_path": "latest"
    },
    "eu_ua_1995": {
        "for_regions": ["eu_ua_1995"],
        "model_path": "latest"
    },
    "eu": {
        "for_regions": ["eu"],
        "model_path": "latest"
    },
    "ru": {
        "for_regions": ["ru", "eu-ua-fake-lnr", "eu-ua-fake-dnr"],
        "model_path": "latest" 
    },
    "kz": {
        "for_regions": ["kz"],
        "model_path": "latest"
    },
    "ge": {
        "for_regions": ["ge"],
        "model_path": "latest"
    }
})

In [7]:
nnet = Detector()
nnet.loadModel(NOMEROFF_NET_DIR)

Config '../NomeroffNet/configs/centermask2/numberplates/cpu_centermask_numberplate_V_39_eSE_FPN_ms_3x.yaml' has no VERSION. Assuming it to be compatible with latest v2.


In [8]:
async def test(dirName, fname):
    img_path = os.path.join(dirName, fname)
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    cv_imgs_masks = nnet.detect_mask([img])
    
    for cv_img_masks in cv_imgs_masks:
        #print(np.array(cv_img_masks).shape)
        # Detect points.
        arrPoints = rectDetector.detect(cv_img_masks)

        # cut zones
        zones = await rectDetector.get_cv_zonesBGR_async(img, arrPoints, 64, 295)

        # find standart
        regionIds, stateIds, countLines = optionsDetector.predict(zones)
        regionNames = optionsDetector.getRegionLabels(regionIds)

        # find text with postprocessing by standart  
        textArr = textDetector.predict(zones, regionNames, countLines)
        textArr = await textPostprocessingAsync(textArr, regionNames)
        return textArr

In [9]:
N = 10

i = 0
j = 0
start_time = time.time()
rootDir = 'images/'
for i in np.arange(N):
    print("pass {}".format(i))
    for dirName, subdirList, fileList in os.walk(rootDir):
        for fname in fileList:
            await test(dirName, fname)
            j += 1
            #print(i, j)
    i += 1
end_time = time.time() - start_time

pass 0
[2.3149750e-04 2.0156287e-04 9.9956685e-01 9.2996437e-08]
[1.4412975e-04 1.3942661e-04 9.9971634e-01 7.0288287e-08]
[1.3142140e-04 1.4232556e-04 9.9972612e-01 6.8887537e-08]
[2.5167220e-04 2.0873037e-04 9.9953949e-01 9.7949673e-08]
[9.9940026e-01 2.5374527e-04 3.4588331e-04 1.2001273e-07]
[1.8707597e-04 1.6363992e-04 9.9964917e-01 8.0326231e-08]
[1.3068340e-04 1.3528616e-04 9.9973387e-01 6.7456774e-08]
[9.9938726e-01 1.8663214e-04 4.2589655e-04 1.1488431e-07]
[2.4314322e-04 2.8550209e-04 9.9947125e-01 1.0761242e-07]
[1.6419563e-04 1.7336561e-04 9.9966228e-01 7.8880063e-08]
[1.5095089e-04 1.4990407e-04 9.9969900e-01 7.2738615e-08]
[3.4934047e-04 6.0885464e-04 9.9904162e-01 1.5634046e-07]
[1.8205604e-01 4.6247598e-03 8.1331688e-01 2.3169791e-06]
[1.6738843e-04 1.4836865e-04 9.9968410e-01 7.5108474e-08]
[1.9555118e-04 2.5081125e-04 9.9955350e-01 9.5746095e-08]
[1.4212406e-04 1.4824526e-04 9.9970943e-01 7.2642045e-08]
[2.2940662e-04 2.3584538e-04 9.9953461e-01 9.7411878e-08]
[1.2808

[1.9390075e-04 2.3298242e-04 9.9957305e-01 9.3293181e-08]
[1.6272889e-04 1.5300966e-04 9.9968421e-01 7.5330433e-08]
[1.4960949e-04 1.6146050e-04 9.9968886e-01 7.5134196e-08]
[9.2288945e-04 2.8321368e-03 9.9624461e-01 3.6091319e-07]
[1.6548210e-04 1.6500994e-04 9.9966943e-01 7.8302463e-08]
[3.5809146e-04 2.6308131e-04 9.9937868e-01 1.1853237e-07]
[1.5103193e-04 1.5399094e-04 9.9969494e-01 7.4006628e-08]
[1.7227540e-04 1.5666838e-04 9.9967098e-01 7.7389885e-08]
[1.7283663e-04 1.7084958e-04 9.9965620e-01 8.0161350e-08]
pass 5
[2.3149750e-04 2.0156287e-04 9.9956685e-01 9.2996437e-08]
[1.4412975e-04 1.3942661e-04 9.9971634e-01 7.0288287e-08]
[1.3142140e-04 1.4232556e-04 9.9972612e-01 6.8887537e-08]
[2.5167220e-04 2.0873037e-04 9.9953949e-01 9.7949673e-08]
[9.9940026e-01 2.5374527e-04 3.4588331e-04 1.2001273e-07]
[1.8707597e-04 1.6363992e-04 9.9964917e-01 8.0326231e-08]
[1.3068340e-04 1.3528616e-04 9.9973387e-01 6.7456774e-08]
[9.9938726e-01 1.8663214e-04 4.2589655e-04 1.1488431e-07]
[2.4314

[1.6738843e-04 1.4836865e-04 9.9968410e-01 7.5108474e-08]
[1.9555118e-04 2.5081125e-04 9.9955350e-01 9.5746095e-08]
[1.4212406e-04 1.4824526e-04 9.9970943e-01 7.2642045e-08]
[2.2940662e-04 2.3584538e-04 9.9953461e-01 9.7411878e-08]
[1.2808184e-04 1.3033889e-04 9.9974149e-01 6.6050667e-08]
[1.7535903e-04 1.9067002e-04 9.9963379e-01 8.4347825e-08]
[1.6956843e-02 2.0503229e-03 9.8099202e-01 8.5313320e-07]
[1.4980708e-04 1.4882268e-04 9.9970120e-01 7.3114890e-08]
[1.9390075e-04 2.3298242e-04 9.9957305e-01 9.3293181e-08]
[1.6272889e-04 1.5300966e-04 9.9968421e-01 7.5330433e-08]
[1.4960949e-04 1.6146050e-04 9.9968886e-01 7.5134196e-08]
[9.2288945e-04 2.8321368e-03 9.9624461e-01 3.6091319e-07]
[1.6548210e-04 1.6500994e-04 9.9966943e-01 7.8302463e-08]
[3.5809146e-04 2.6308131e-04 9.9937868e-01 1.1853237e-07]
[1.5103193e-04 1.5399094e-04 9.9969494e-01 7.4006628e-08]
[1.7227540e-04 1.5666838e-04 9.9967098e-01 7.7389885e-08]
[1.7283663e-04 1.7084958e-04 9.9965620e-01 8.0161350e-08]


### Intel(R) Core(TM) i9-9900K CPU @ 3.60GHz 

In [10]:
print(f"Processed {j} photos")
print(f"Time {end_time}")
print(f"One photo process {end_time/j} seconds")

Processed 260 photos
Time 233.77865839004517
One photo process 0.8991486861155583 seconds


### Intel(R) Core(TM) i9-9900K CPU @ 3.60GHz + Quadro RTX 8000